In [1]:
from agents import Agent, Runner, trace, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.mcp import MCPServerStdio

from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv(override=True)

True

# Accounts module

In [4]:
from accounts import Account

In [5]:
account = Account.get('Ed')
account

Account(name='ed', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [6]:
account.buy_shares('AMZN', 1, 'This bookstore website looks promising')

'Completed. Latest details:\n{"name": "ed", "balance": 9994.99, "strategy": "", "holdings": {"AMZN": 1}, "transactions": [{"symbol": "AMZN", "quantity": 1, "price": 5.01, "timestamp": "2025-08-05 16:13:09", "rationale": "This bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-05 16:13:09", 10054.99]], "total_portfolio_value": 10054.99, "total_profit_loss": 54.98999999999978}'

In [7]:
account.report()

'{"name": "ed", "balance": 9994.99, "strategy": "", "holdings": {"AMZN": 1}, "transactions": [{"symbol": "AMZN", "quantity": 1, "price": 5.01, "timestamp": "2025-08-05 16:13:09", "rationale": "This bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-05 16:13:09", 10054.99], ["2025-08-05 16:13:11", 10015.99]], "total_portfolio_value": 10015.99, "total_profit_loss": 15.989999999999782}'

In [8]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 1,
  'price': 5.01,
  'timestamp': '2025-08-05 16:13:09',
  'rationale': 'This bookstore website looks promising'}]

# Accounts MCP Server

In [15]:
params = {"command": "uv", "args": ["run", "accounts_server.py", "--active"]}  # --active to target currently activated virtual env

async with MCPServerStdio(params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\nArgs:\n    name: name of the account holder\n    symbol: symbol of the stock\n    quantity: how many shares to buy\n    rationale: reason for purchasing, fit with the account's strategy", inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}, 'symbol': {'title': 'Symbol', 'type': 'string'}, 'quantity': {'title': 'Quantity', 'type': 'integer'}, 'rationale': {'title': 'Rationale', 'type': 'string'}}, 'required'

In [16]:
import os

gemini_client = AsyncOpenAI(base_url=os.environ['GEMINI_BASE_URL'], api_key=os.environ['GEMINI_API_KEY'])
gemini_model = OpenAIChatCompletionsModel(model='gemini-2.5-flash', openai_client=gemini_client)

In [17]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My account is under the name Ed. What's my balance and holdings?"

In [19]:
async with MCPServerStdio(params, client_session_timeout_seconds=10) as server:
    agent = Agent(
        name='account_manager',
        instructions=instructions,
        model=gemini_model,
        mcp_servers=[server]
    )

    with trace('account_manager'):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Your balance is 9994.99 and you hold 1 share of AMZN.